## Mojo optimisation strategy: Only write mutated fields of a large struct ##  

If you are looping through a list and mutating the value of a single field in a `struct`, try **only writing to the mutated field in the struct** when putting it back in the list.  

This optimisation was discovered while playing with [this N-body implementation](https://github.com/paugier/nbabel/blob/main/mojo/mikowals_experiments.mojo#L66-L84).  The filename contains my username because I made some initial tweaks that sped the code up and the repository owner wanted to preserve them and keep them separate from other work.  

Discovered may not be the right word.  To anyone who has ever worked with a database, updating only mutated fields is an obvious way to move less data and consume less compute.  For Mojo though it is less obvious why that would work.  It does seem important in at least some cases though and I have not seen this behaviour explained or discussed anywhere.

I am still not sure of all the elements that combine to make this optimisation necessary.  I also have basic questions about how and why it works.

That said, below are some steps to create a contrived scenario where updating a single field gets a dramatic improvement.  As new versions of Mojo are released I can rerun the code and watch for changes.

#### Make a simple but somewhat large struct ####
If the `struct` only contains a few values it doesn't matter how it is updated and it will be fast.  For example, trying this optimisation had no impact on Modular's own [nbody.mojo](https://github.com/modularml/mojo/blob/main/examples/nbody.mojo) example.  That code has three fields and each holds a SIMD vector with four float64 values.  For this demonstration, I have gone crazy and used seven fields with vectors of sixteen values each.

But it is still very simple.  I use the `@value` decorator to generate boilerplate code for moving and copying.  I could also make it `@register_passable` or `@register_passable("trivial")` and it shows the same benefit from this optimisation.  There is nothing special happening here other than making the stuct a bit bulky.

In [2]:
from random import random_float64
# Used to fill SIMD with values.  Vectors of zero values might be fast no matter what.
fn create_rand_SIMD16() -> SIMD[DType.float64, 16]:
    var res = SIMD[DType.float64, 16](0.0)
    for i in range(16):
        res[i] = random_float64()
    return res

@value
struct ManyFields(CollectionElement):
    var a: SIMD[DType.float64, 16]
    var b: SIMD[DType.float64, 16]
    var c: SIMD[DType.float64, 16]
    var d: SIMD[DType.float64, 16]
    var e: SIMD[DType.float64, 16]
    var f: SIMD[DType.float64, 16]
    var g: SIMD[DType.float64, 16]

    fn __init__(inout self):
        self.a = create_rand_SIMD16()
        self.b = create_rand_SIMD16()
        self.c = create_rand_SIMD16()
        self.d = create_rand_SIMD16()
        self.e = create_rand_SIMD16()
        self.f = create_rand_SIMD16()
        self.g = create_rand_SIMD16()

#### Write a slow function ###
The math below is nonsense.  I tried writing simpler nonsense. For example, I accumulated all the other fields into `a` with some division to prevent overflow.  But that code was fast.  So in the end I did a series of steps using several values from the two structs. 

And with a large enough number of fields being read or many intermediate values computed, the code could get really slow depending on how elements were written to the heap.  Here it is done the slow way. Writing back the entire `struct` to the vector in the last two lines.(see note 1 below)

I also use a nested loop structure so we don't need many elements to get a lot of writes happening.  The loops can be unrolled and it will be faster but benefit of this optimisation persists.  
__________


Note 1: I have strong doubts that this optimisation works anything at all like my initial intuition or how I have explained it so far.  My intuition was that writing one field to the heap is more efficient than writing many fields.  And doing a change that makes it look like you are writing one field is certainly faster.  But do we really know the address of the one field on the heap so we can find it and write it quickly?

If you modify the `struct` to print each time `__copyinit__` and `__moveinit__` are called, you can see we are always working with the whole `struct`.  The code in our `fast_mutating_loop` (a few sections down) does look like it is writing one field.  The trouble is that `__copyinit__` is called on the whole struct that will be replaced.  The mutated field is updated in that copy.  The no longer needed item-that-will-be-replaced is deleted. And finally `__moveinit__` places the now mutated element back into the vector.  Since `slow_mutating_loop` below already got a copy and mutated the desired field, it should be faster just to move that existing copy to the vector.  Which it could do because they are local copies that are not used anywhere else.  But just moving the mutated element into the vector is slower.

Any one who can write a correct explanation for why this optimisation works is most welcome.  But for now my intuitive explanation seems like the best way to make it a memorable thing to look for.

In [3]:
fn slow_mutating_loop(inout v: DynamicVector[ManyFields]):
    for i in range(len(v)):
        var mf_i = v[i]
        
        for j in range(i + 1, len(v)):
            var mf_j = v[j]
            # Irrelevant math loosely based on the N-body problem
            let delta = mf_i.g - mf_j.g
            let d_sq = delta * delta
            let mag = random_float64() / d_sq
            mf_j.a += (mf_i.b + mf_i.c) * mag
            mf_i.a += (mf_j.b + mf_j.c) * mag
            # End of irrelevant math. 
            
            # We have only changed `a` but we are writing the entire struct.
            v[j] = mf_j

        # Same mistake again but in an outer loop so the slow down is barely noticeable.
        v[i] = mf_i

#### Create a benchmark function ####
This is a standard Mojo way to time and compare functions.  It relies on Mojo's Benchmark module which has very useful features to do a few warm ups before timing, run a lot of iterations in the time it is alloted, and produce a nice report of what it did.

One notable thing here is that I use a DynamicVector to hold the elements.  There are a few other choices that could be made but as far as I could tell which collection type I chose was irrelevant to the demonstration.  I tried InlinedFixedVector, StaticTuple, Pointer, and a custom struct.  This optimisation appears to work for all of them.  

In [4]:
from benchmark import run 

fn bench[name: String, func: fn(inout DynamicVector[ManyFields]) -> None, size: Int]() -> Float64:

    # create and fill a vector
    var v = DynamicVector[ManyFields](size)
    for i in range(size):
        v.append(ManyFields())
    
    @parameter
    fn wrapper():
        func(v)
        
    # the actual timing
    let result = run[wrapper](min_runtime_secs=0.01, max_runtime_secs=1.0)
    
    # prevent v being destroyed early
    _ = (v,)
    
    print(name, "updated", size, "elements in", result.mean(), "seconds.")
    return result.mean()

#### Time the slow function ####

In [5]:
let slow_time = bench["Slow", slow_mutating_loop, 512]()

Slow updated 512 elements in 0.003714698412698413 seconds.


#### The fast version ####
We now change the heap updates so they look like they only handle a single field.  Please read 'note 1' above. The change is simple and it looks intuitive.  But why it works is not really clear. 

The key change is `v[j] = mf_j` becomes `v[j].a = mf_j.a`.

The same change in the outer loop also makes a measurable difference, but nothing like the inner loop since it is only called once for each element.

In [6]:
fn fast_mutating_loop(inout v: DynamicVector[ManyFields]):
    for i in range(len(v)):
        var mf_i = v[i]
        
        for j in range(i + 1, len(v)):
            var mf_j = v[j]
            # Same irrelevant math loosely based on the N-body problem
            let delta = mf_i.g - mf_j.g
            let d_sq = delta * delta
            let mag = random_float64() / d_sq
            mf_j.a += (mf_i.b + mf_i.c) * mag
            mf_i.a += (mf_j.b + mf_j.c) * mag
            
            # New fast line to write to the heap
            v[j].a = mf_j.a

        # New fast write to the heap here also.
        v[i].a = mf_i.a

#### Time the fast version ####

In [7]:
let fast_time = bench["Fast", fast_mutating_loop, 512]()
print("A speedup of", slow_time / fast_time, "times.")

Fast updated 512 elements in 0.0020175977011494253 seconds.
A speedup of 1.8411492095684634 times.


#### Results ####
So those small changes almost make the code twice as fast!  The fast version of the code is slightly less readable. And hopefully my necessarily elaborate demonstration makes it obvious that most code will not get any benefit from this change.   But with a 1.8x speedup possible it is well worth looking for opportunities where only one field is changing.  And I have not yet seen an instance where trying to change one field made the code slower.  But it is ugly to read.

Ideally this will help identify a bug.  If that happens I will update the timings here and explain that.  That seems more ideal to me than for this style of writing updates to become standard.  But I don't know if it is a bug.

My actual guess of what is happening is that the large struct and many math operations have an impact on either what is in caches or the compiler's ability to determine that only one field changed.  Somehow writing as if we are updating a single field gets around that by copying the element out, changing one field, and immediately putting most of the same values back.  I say this because:
- DynamicVector appears to make an extra copy and delete in the fast version.  "Extra copy" sounds like something to be avoided but copying just before the write is an optimisation.
- `register_passable` appears to have no impact.  The struct may be too big to utilise being passed in registers effectively. 
- Simple math and a few calculations did not reproduce the slow path. Loading lots of values and saving intermediate values would change what is in the cache between reading from the vector and writing to it. 

But I am well out of my depth in guessing what is actually happening.

## The End ##

#### The stuff below is unnecessary fun and loosely related observations ####

#### Fun parametric version to make sure math is identical ####

In [8]:
fn mutating_loop[make_fast: Bool = False](inout v: DynamicVector[ManyFields]):
    for i in range(len(v)):
        var mf_i = v[i]
        for j in range(i + 1, len(v)):
            var mf_j = v[j]
            let delta = mf_i.g - mf_j.g
            let d_sq = delta * delta
            let mag = random_float64() / d_sq
            mf_j.a += (mf_i.b + mf_i.c) * mag
            mf_i.a += (mf_j.b + mf_j.c) * mag
            if make_fast:
                v[j].a = mf_j.a
            else:
                v[j] = mf_j
        
        if make_fast:
            v[i].a = mf_i.a
        else:
            v[i] = mf_i

#### Fancy parametric reproduction of results ####

In [9]:
let another_slow_time = bench["Slow", mutating_loop[make_fast = False], 512]()
let another_fast_time = bench["Fast", mutating_loop[make_fast = True], 512]()
print("A speedup of", another_slow_time / another_fast_time, "times.")

Slow updated 512 elements in 0.0037189523809523809 seconds.
Fast updated 512 elements in 0.002011279569892473 seconds.
A speedup of 1.849047957639824 times.


#### Demonstrate the extra copy in our fast version ####
For a new language like Mojo it is fun to see how it is working.  This particular behaviour may not last long as it should be more efficient to move the old element, update it, and move back the changed element.  Such a change might break the fast version above.

To demonstrate, I will write a simple struct with one field.  The struct is Verbose and prints when it is moved, copied or deleted.  I will then use the two different update methods: 1) whole element, and 2) single field.  

When writing the whole element, what I see is what I expect:
- old element moved (for deletion)
- old element deleted
- new element moved into the vector

When writing a single field, I expected to see two moves but instead you get:
- old element copied
- old element moved (for deletion)
- old element deleted
- new element moved

In [10]:
struct Verbose(CollectionElement):
    var value: Int16

    fn __init__(inout self, value: Int16):
        self.value = value

    fn __copyinit__(inout self, other: Self):
        print("copyinit from value:", other.value)
        self.value = other.value

    fn __moveinit__(inout self, owned other: Self):
        print("moveinit from value:", other.value)
        self.value = other.value ^
    
    fn __del__(owned self):
        print("deleting value:", self.value)
        # nothing to be done since all fields have a __del__ method

fn demonstrate_moves_and_copies():
    #fill array
    var v = DynamicVector[Verbose](4)
    for i in range(4):
        v.append(Verbose(i))

    print("done filling array")
    print()

    print("write a whole element at index 0")
    v[0] = Verbose(10)
    print("done writing a whole element")
    print()

    print("write one field at index 1")
    v[1].value = 20
    print("done writing one field")
    print()

    #keep the DynamicVector around so we don't get extra deletes above.
    print("watch DynamicVector clean up by moving and deleting each element.")
    _ = (v,)

demonstrate_moves_and_copies()

moveinit from value: 0
moveinit from value: 1
moveinit from value: 2
moveinit from value: 3
done filling array

write a whole element at index 0
moveinit from value: 0
deleting value: 0
moveinit from value: 10
done writing a whole element

write one field at index 1
copyinit from value: 1
moveinit from value: 1
deleting value: 1
moveinit from value: 20
done writing one field

watch DynamicVector clean up by moving and deleting each element.
moveinit from value: 10
deleting value: 10
moveinit from value: 20
deleting value: 20
moveinit from value: 2
deleting value: 2
moveinit from value: 3
deleting value: 3


#### What is single field write doing? And how? ####
Maybe this is obvious to everyone else but it wasn't obvious to me.  `v[i].a = mf_j.a` works by calling `__setitem__`.  It appears to be a feature added into the `[i]` shorthand version which usually can be thought of as `__setitem__(i)`.  But if you try `v.__setitem__(i).a = mf_j.a` you get errors.  So that shorthand may be an important part of what I am writing about.

I absolutely expect that `v[i].a` would lead to the element at `i` getting its `a` field updated and all other fields unchanged.  And that is what happens.  But how to manually manage it with my own `__setitem__` is a mystery.  Also, I am pretty sure based on the extra copy in DynamicVector above that `v[i].a = whatever` is misleading.  I don't think many people would read that code as "write a new element to v[i] but do it by copying the old element to a new element and then delete the old element". That it runs faster in the code above is odd.  It has an extra copy and an extra delete.  Move element, update it, put it back.  No copy or delete.  

But without the copy is the slow path. *** mind blows up ***